## Import Libraries

In [254]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [255]:
import matplotlib.pyplot as plt
plt.ion()

In [256]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [257]:
from os import getcwd, path, pardir
import pickle

## Compile and program project

In [258]:
project_path = [path.curdir, "AES_Flash-S"]
#project_path = 

In [259]:
atprogram("./AES_FLASH-S",device_name = "ATSAML11E16A" ,verbose=3)

Building file: ../../../../shared/mbedtls/aesni.c
Invoking: ARM/GNU C Compiler : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\bin\arm-none-eabi-gcc.exe"  -x c -mthumb -D__SAML11E16A__ -DDEBUG  -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\arm\cmsis\5.0.1\CMSIS\Include" -I"../../../../shared" -I"../Config" -I".." -I"../examples" -I"../hal/include" -I"../hal/utils/include" -I"../hpl/core" -I"../hpl/crya" -I"../hpl/dmac" -I"../hpl/gclk" -I"../hpl/mclk" -I"../hpl/nvmctrl" -I"../hpl/osc32kctrl" -I"../hpl/oscctrl" -I"../hpl/pm" -I"../hpl/port" -I"../hri" -I"../trustzone" -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\atmel\SAML11_DFP\1.0.109\include"  -O3 -ffunction-sections -mlong-calls -g3 -Wall -mcpu=cortex-m23 -c -std=gnu99 -mcmse -MD -MP -MF "mbedtls/aesni.d" -MT"mbedtls/aesni.d" -MT"mbedtls/aesni.o"   -o "mbedtls/aesni.o" "../../../../shared/mbedtls/aesni.c" 
Finished building: ../../../../shared/mbedtls/aesni.c
Building file: ../../../../shared/m

0

In [260]:
#atprogram(path.abspath(path.join(*project_path)), verbose=2)

## Data Logging

In [261]:
live_plot = False

Create a figure for the plot.

In [262]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [263]:
config_dict = {
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "file_name_base": "experiment_aes_flash"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_method": "line",
    "plot_xmax": 5,
    "window_title": "Experiment AES-128 Flash",
}

Stop criteria to pass to the logger:

In [264]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Perform the measurement.

In [265]:
data = []
cd = config_dict.copy()
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
    
    cd.update(config_dict_plot)
    cd["fig"] = fig
    
with DGILibExtra(**cd) as dgilib:
    dgilib.device_reset()
    dgilib.logger.log(1000,stop_fn)
    data = dgilib.data

In [266]:
data.length()

{48: 6006, 256: 1787000}

# Store Data

In [267]:
import pickle

In [268]:
pickle.dump(data, open("aes_flash_logger_data.p", "wb"))

# Load Data

In [269]:
data = pickle.load(open("aes_flash_logger_data.p", "rb"))

In [270]:
iteration = 0
name = "AES-128_Flash"
#data = pickle.load(open(path.join(path.pardir, path.pardir, f"{name}_{iteration}.p"), "rb"))

## Analysis

Create Stop Function to stop parsing the data when all pins are high.

In [271]:
def stop_function(pin_values):
    return all(pin_values)

Parse the data.

In [272]:
aes_charge, aes_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)

In [273]:
flash_charge, flash_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

In [274]:
print(len(aes_charge), len(aes_time), len(flash_charge), len(flash_time))
# cutoff = min(len(aes_charge), len(aes_time), len(flash_charge), len(flash_time))
# aes_charge = aes_charge[:cutoff]
# aes_time = aes_time[:cutoff]
# flash_charge = flash_charge[:cutoff]
# flash_time = flash_time[:cutoff]

750 750 750 750


In [275]:
# length = len(aes_charge)
# assert length == len(aes_time)
# assert length == len(flash_charge)
# assert length == len(flash_time)
# print(length)

In [276]:
aes_encrypt_charge = aes_charge[0::2]
aes_decrypt_charge = aes_charge[1::2]
aes_encrypt_time = aes_time[0::2]
aes_decrypt_time = aes_time[1::2]

In [277]:
aes_flash_write_charge = flash_charge[0::2]
aes_flash_read_charge = flash_charge[1::2]
aes_flash_write_time = flash_time[0::2]
aes_flash_read_time = flash_time[1::2]

In [278]:
len(aes_encrypt_charge), len(aes_decrypt_charge), len(aes_encrypt_time), len(aes_decrypt_time), len(aes_flash_write_charge), len(aes_flash_read_charge), len(aes_flash_write_time), len(aes_flash_read_time)

(375, 375, 375, 375, 375, 375, 375, 375)

In [279]:
drop = 0
cutoff = min(len(aes_encrypt_charge), len(aes_decrypt_charge), len(aes_encrypt_time), len(aes_decrypt_time), len(aes_flash_write_charge), len(aes_flash_read_charge), len(aes_flash_write_time), len(aes_flash_read_time)) - drop
aes_encrypt_charge = aes_encrypt_charge[:cutoff]
aes_decrypt_charge = aes_decrypt_charge[:cutoff]
aes_encrypt_time = aes_encrypt_time[:cutoff]
aes_decrypt_time = aes_decrypt_time[:cutoff]
aes_flash_write_charge = aes_flash_write_charge[:cutoff]
aes_flash_read_charge = aes_flash_read_charge[:cutoff]
aes_flash_write_time = aes_flash_write_time[:cutoff]
aes_flash_read_time = aes_flash_read_time[:cutoff]

In [280]:
length = len(aes_encrypt_charge)
assert length == len(aes_decrypt_charge)
assert length == len(aes_encrypt_time)
assert length == len(aes_decrypt_time)
assert length == len(aes_flash_write_charge)
assert length == len(aes_flash_read_charge)
assert length == len(aes_flash_write_time)
assert length == len(aes_flash_read_time)
print(length)

375


# Convert to Joule

In [281]:
voltage = 3.33
j_scale = 1e3 # m
t_scale = 1e3 # m
model_j_scale = 1e6 # n
model_t_scale = 1e3 # u

experiment_name = 'AES-128'

In [282]:
aes_encrypt_energy = aes_encrypt_charge[:cutoff]
aes_flash_write_energy = aes_flash_write_charge[:cutoff]
aes_flash_read_energy = aes_flash_read_charge[:cutoff]
aes_decrypt_energy = aes_decrypt_charge[:cutoff]
aes_encrypt_time_s = aes_encrypt_time[:cutoff]
aes_flash_write_time_s = aes_flash_write_time[:cutoff]
aes_flash_read_time_s = aes_flash_read_time[:cutoff]
aes_decrypt_time_s = aes_decrypt_time[:cutoff]

for i in range(len(aes_encrypt_energy)):
    aes_encrypt_energy[i] = aes_encrypt_energy[i] * voltage * j_scale
for i in range(len(aes_flash_write_energy)):
    aes_flash_write_energy[i] = aes_flash_write_energy[i] * voltage * j_scale
for i in range(len(aes_flash_read_energy)):
    aes_flash_read_energy[i] = aes_flash_read_energy[i] * voltage * j_scale
for i in range(len(aes_decrypt_energy)):
    aes_decrypt_energy[i] = aes_decrypt_energy[i] * voltage * j_scale
    
for i in range(len(aes_encrypt_time_s)):
    aes_encrypt_time_s[i] = aes_encrypt_time_s[i] * t_scale
for i in range(len(aes_flash_write_time_s)):
    aes_flash_write_time_s[i] = aes_flash_write_time_s[i] * t_scale
for i in range(len(aes_flash_read_time_s)):
    aes_flash_read_time_s[i] = aes_flash_read_time_s[i] * t_scale
for i in range(len(aes_decrypt_time_s)):
    aes_decrypt_time_s[i] = aes_decrypt_time_s[i] * t_scale

In [283]:
MBEDTLS_AES_BLOCK_SIZE = 16
STEP_SIZE = MBEDTLS_AES_BLOCK_SIZE
MIN_NUM_BYTES = STEP_SIZE
num_bytes = range(MIN_NUM_BYTES, MIN_NUM_BYTES + STEP_SIZE * len(aes_encrypt_energy), STEP_SIZE)
print(f"MAX_NUM_BYTES: {num_bytes[-1]}")

MAX_NUM_BYTES: 6000


In [284]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)
# pars['intercept'].set(min=0)

In [285]:
results = []
ylabels = (['Energy [mJ]'] * 2 + ['Time [ms]'] * 2) * 2 + ['Energy [mJ]'] + ['Time [ms]']
parameter_names = [
    'Encrypt Energy',
    'Flash Write Energy',
    'Flash Read Energy',
    'Decrypt Energy',
    'Encrypt Time',
    'Flash Write Time',
    'Flash Read Time',
    'Decrypt Time',
    'Total Energy',
    'Total Time',
]
for y in [aes_encrypt_energy, aes_flash_write_energy, aes_flash_read_energy, aes_decrypt_energy, aes_encrypt_time_s, aes_flash_write_time_s, aes_flash_read_time_s, aes_decrypt_time_s,
          [e + w + r + d for (e,w,r,d) in zip(aes_encrypt_energy, aes_flash_write_energy, aes_flash_read_energy, aes_decrypt_energy)],
          [e + w + r + d for (e,w,r,d) in zip(aes_encrypt_time_s, aes_flash_write_time_s, aes_flash_read_time_s, aes_decrypt_time_s)]]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    fig, grid = result.plot(
        xlabel='Checkpoint Size [Bytes]',
        ylabel=ylabels[len(results)])
    fig.tight_layout(rect=(0.05, 0.05, 1, 1))
    fig.set_size_inches(5, 4.5, forward=True)
    fig.canvas.set_window_title(
        f"Residuals of {experiment_name} {parameter_names[len(results)]}")
    fig.show()
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 375
    # variables        = 2
    chi-square         = 9.3308e-06
    reduced chi-square = 2.5016e-08
    Akaike info crit   = -6561.91903
    Bayesian info crit = -6554.06518
[[Variables]]
    slope:      7.1836e-05 +/- 4.7155e-09 (0.01%) (init = 0)
    intercept: -2.6078e-04 +/- 1.6368e-05 (6.28%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.867
[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 6
    # data points      = 375
    # variables        = 2
    chi-square         = 0.00813012
    reduced chi-square = 2.1797e-05
    Akaike info crit   = -4023.16450
    Bayesian info crit = -4015.31065
[[Variables]]
    slope:      8.2428e-05 +/- 1.3919e-07 (0.17%) (init = 0)
    intercept:  0.01306170 +/- 4.8315e-04 (3.70%) (init = 1)
[[Correlations]] (unreport

In [286]:
fig2 = plt.figure(figsize=(8, 6))
fig2.canvas.set_window_title(f"Analysis {experiment_name}")

In [287]:
charge_color = 'r'
time_color = 'b'

In [288]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Checkpoint Size [Bytes]')
ax1.set_ylabel('Energy [mJ]', color=charge_color)
ax2.set_ylabel('Time [ms]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [289]:
lines = []
lines += ax1.plot(num_bytes, aes_encrypt_energy, charge_color+'-', label=f'{parameter_names[len(lines)]}')
lines += ax1.plot(num_bytes, aes_flash_write_energy, charge_color+'-.', label=f'{parameter_names[len(lines)]}')
lines += ax1.plot(num_bytes, aes_flash_read_energy, charge_color+':', label=f'{parameter_names[len(lines)]}')
lines += ax1.plot(num_bytes, aes_decrypt_energy, charge_color+'--', label=f'{parameter_names[len(lines)]}')
lines += ax2.plot(num_bytes, aes_encrypt_time_s, time_color+'-', label=f'{parameter_names[len(lines)]}')
lines += ax2.plot(num_bytes, aes_flash_write_time_s, time_color+'-.', label=f'{parameter_names[len(lines)]}')
lines += ax2.plot(num_bytes, aes_flash_read_time_s, time_color+':', label=f'{parameter_names[len(lines)]}')
lines += ax2.plot(num_bytes, aes_decrypt_time_s, time_color+'--', label=f'{parameter_names[len(lines)]}')
ax1.legend(handles=lines)
ax1.set_title(
    f"{parameter_names[0]}: Slope {results[0].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[0].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[1]}: Slope {results[1].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[1].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[2]}: Slope {results[2].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[2].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[3]}: Slope {results[3].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[3].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[4]}: Slope {results[4].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[4].params['intercept'].value * model_t_scale:.04} $\mu$s\n" +
    f"{parameter_names[5]}: Slope {results[5].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[5].params['intercept'].value * model_t_scale:.04} $\mu$s\n" +
    f"{parameter_names[6]}: Slope {results[6].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[6].params['intercept'].value * model_t_scale:.04} $\mu$s\n" +
    f"{parameter_names[7]}: Slope {results[7].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[7].params['intercept'].value * model_t_scale:.04} $\mu$s\n" +
    f"{parameter_names[8]}: Slope {results[8].params['slope'].value * model_j_scale:.04} nJ/B, Intercept {results[8].params['intercept'].value * model_j_scale:.04} nJ\n" +
    f"{parameter_names[9]}: Slope {results[9].params['slope'].value * model_t_scale:.04} $\mu$s/B, Intercept {results[9].params['intercept'].value * model_t_scale:.04} $\mu$s\n")
fig2.tight_layout()
fig2.show()

In [290]:
print(
    f"{parameter_names[0]}: Slope {results[0].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[0].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[1]}: Slope {results[1].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[1].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[2]}: Slope {results[2].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[2].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[3]}: Slope {results[3].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[3].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[4]}: Slope {results[4].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[4].params['intercept'].value * model_t_scale:.020} $\mu$s\n" +
    f"{parameter_names[5]}: Slope {results[5].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[5].params['intercept'].value * model_t_scale:.020} $\mu$s\n" +
    f"{parameter_names[6]}: Slope {results[6].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[6].params['intercept'].value * model_t_scale:.020} $\mu$s\n" +
    f"{parameter_names[7]}: Slope {results[7].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[7].params['intercept'].value * model_t_scale:.020} $\mu$s\n" +
    f"{parameter_names[8]}: Slope {results[8].params['slope'].value * model_j_scale:.020} nJ/B, Intercept {results[8].params['intercept'].value * model_j_scale:.020} nJ\n" +
    f"{parameter_names[9]}: Slope {results[9].params['slope'].value * model_t_scale:.020} $\mu$s/B, Intercept {results[9].params['intercept'].value * model_t_scale:.020} $\mu$s\n"
)

Encrypt Energy: Slope 71.836039741738460407 nJ/B, Intercept -260.78262044637591544 nJ
Flash Write Energy: Slope 82.427526726457500672 nJ/B, Intercept 13061.702200219182487 nJ
Flash Read Energy: Slope 2.3851581782099495754 nJ/B, Intercept -11.377969016956868131 nJ
Decrypt Energy: Slope 71.56735718308297578 nJ/B, Intercept -261.21935008771578168 nJ
Encrypt Time: Slope 39.939523854176144368 $\mu$s/B, Intercept 1.6174011305954705264 $\mu$s
Flash Write Time: Slope 12.43858430131312609 $\mu$s/B, Intercept 1857.3069294673116474 $\mu$s
Flash Read Time: Slope 1.9990560784886945989 $\mu$s/B, Intercept 7.8095544945304684603 $\mu$s
Decrypt Time: Slope 40.001000679920515779 $\mu$s/B, Intercept 6.0027394445997437344 $\mu$s
Total Energy: Slope 228.21608182948867238 nJ/B, Intercept 12528.322267213010491 nJ
Total Time: Slope 94.378164605781975638 $\mu$s/B, Intercept 1872.7378582935748454 $\mu$s



In [291]:
# Save Charge amount list into pickle file
import pickle
pickle.dump(aes_encrypt_energy, open("aes_flash_encrypt_energy_mJ.p", "wb"))
pickle.dump(aes_decrypt_energy, open("aes_flash_decrypt_energy_mJ.p", "wb"))
pickle.dump(aes_flash_write_energy, open("aes_flash_write_energy_mJ.p", "wb"))
pickle.dump(aes_flash_read_energy, open("aes_flash_read_energy_mJ.p", "wb"))
pickle.dump(aes_encrypt_time_s, open("aes_flash_encrypt_time_ms.p", "wb"))
pickle.dump(aes_decrypt_time_s, open("aes_flash_decrypt_time_ms.p", "wb"))
pickle.dump(aes_flash_write_time_s, open("aes_flash_write_time_ms.p", "wb"))
pickle.dump(aes_flash_read_time_s, open("aes_flash_read_time_ms.p", "wb"))

In [292]:
aes = [aes_encrypt_energy, aes_flash_write_energy, aes_flash_read_energy, aes_decrypt_energy, aes_encrypt_time_s, aes_flash_write_time_s, aes_flash_read_time_s, aes_decrypt_time_s]
for i in aes:
    print(len(i), len(i)*16)

375 6000
375 6000
375 6000
375 6000
375 6000
375 6000
375 6000
375 6000


## Write config file

In [293]:
import json

config = {}
config["name"] = "AES-128 Flash"
config["project_paths"] = [project_path]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["AES-128 Encrypt", "AES-128 Decrypt"], 3: ["AES-128 Flash Write", "AES-128 Flash Read"]}, 
                      "result_types": ["Charge", "Time"],
                      "section_types": {"init": [], 
                                        "store": ["AES-128 Encrypt", "AES-128 Flash Write"],
                                        "load": ["AES-128 Flash Read", "AES-128 Decrypt"],
                                        "exit": []},
                      "labels": {
                          "Charge": {"x":"Data Size", "x_unit": "byte", "y": "Charge", "y_unit": "C"},
                          "Time": {"x":"Data Size", "x_unit": "byte", "y": "Time", "y_unit": "s"},
                      },
                      "x_step": MBEDTLS_AES_BLOCK_SIZE}

with open("looped_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)

# Write model data

In [294]:
dump_pickle = True
fit_lm = True
verbose = 2
show_lm_plot = 2
# drop = 1

In [295]:
# Parse data
analysis_config = config.get("analysis")
result_types = analysis_config.get("result_types")
x_step = analysis_config.get("x_step")
parsed_data = {}
for pin, parameter_names in analysis_config.get("pins").items():
    data2 = power_and_time_per_pulse(
        data, int(pin), stop_function=stop_function)
    num_names = len(parameter_names)
    for i, parameter_name in enumerate(parameter_names):
        end_index = -drop * num_names or None
        parsed_data[parameter_name] = {
            result_types[0]: data2[0][i:end_index:num_names],
            result_types[1]: data2[1][i:end_index:num_names],
            "x_step": x_step}
if dump_pickle:
    pickle.dump(parsed_data, open(
        path.join(path.curdir,
                  f"{config_dict.get('file_name_base')}_looped.p"), "wb"))

In [296]:
# Fit lm
if fit_lm:
    model = None
    if model is None:
        def line(x, intercept, slope):
            """a line"""
            return [intercept + slope*i for i in x]

        model = Model(line)
        params = model.make_params(intercept=0, slope=1)
#         params['intercept'].set(min=0)
    else:
        params = model.params

    model_results = {}
    labels = analysis_config.get("labels")
    for parameter_name in parsed_data.keys():
        length = len(parsed_data[parameter_name][result_types[0]])
        x_step = parsed_data[parameter_name]["x_step"]
        num_bytes = range(x_step, (length+1)*x_step, x_step)
        if verbose:
            print(
                f"Fitting model to {parameter_name} with {length} " +
                f"samples, from {min(num_bytes)} to {max(num_bytes)} "
                f"bytes in steps of {x_step}.")
        model_result = {}
        for result_type in result_types:
            model_result[result_type] = model.fit(
                parsed_data[parameter_name][result_type], params,
                x=num_bytes)
            if verbose >= 2:
                print(model_result[result_type].fit_report())
            # Plot multiple view
            if show_lm_plot >= 2:
                fig, grid = model_result[result_type].plot(
                    xlabel=f"{labels[result_type]['x']} " +
                           f"[{labels[result_type]['x_unit']}]",
                    ylabel=f"{labels[result_type]['y']} " +
                           f"[{labels[result_type]['y_unit']}]")
                fig.canvas.set_window_title(
                    f"Residuals of {parameter_name}")
                fig.tight_layout()
                fig.show()
            model_results[parameter_name] = model_result

    # Plot single view
    if show_lm_plot:
        import matplotlib.pyplot as plt
        fig = plt.figure(figsize=(9, 6))
        fig.canvas.set_window_title(f"Analysis {config.get('name')}")
        colors = dict(zip(result_types, ['r', 'b']))
        line_styles = (
            line_style for line_style in ('-', '--', '-.', ':') * 2)
        # fig.suptitle(f"Energy analysis of {config.get('name')}")
        ax = {}
        ax[result_types[0]] = fig.add_subplot(1, 1, 1)
        ax[result_types[1]] = ax[result_types[0]].twinx()
        ax[result_types[0]].set_xlabel(
            f"{labels[result_types[0]]['x']} " +
            f"[{labels[result_types[0]]['x_unit']}]")
        for result_type in result_types:
            ax[result_type].set_ylabel(
                f"{labels[result_type]['y']} " +
                f"[{labels[result_type]['y_unit']}]",
                color=colors[result_type])
            ax[result_type].tick_params('y', colors=colors[result_type])

        lines = []
        title_str = ""
        for parameter_name in parsed_data.keys():
            length = len(parsed_data[parameter_name][result_types[0]])
            x_step = parsed_data[parameter_name]["x_step"]
            num_bytes = range(x_step, (length+1)*x_step, x_step)
            model_result = {}
            line_style = next(line_styles)
            for result_type in result_types:
                label = f"{parameter_name} {labels[result_type]['y']}"
                lines += ax[result_type].plot(
                    num_bytes, parsed_data[parameter_name][result_type],
                    colors[result_type] + line_style, label=label)
                title_str += f"{label} "
                for param in params.keys():
                    title_str += "".join(
                        f"{params[param].name.capitalize()}: ")
                    title_str += "".join(
                        f"{model_results[parameter_name][result_type].params[param].value: .03} ")
                    title_str += "".join(
                        f"{labels[result_type]['y_unit']}, ")
                title_str = title_str[:-2] + \
                    f" per {labels[result_type]['x_unit']}\n"
        ax[result_types[0]].legend(handles=lines)
        ax[result_types[0]].set_title(title_str[:-1])
        # fig.tight_layout()
        fig.tight_layout(rect=(0.05, 0.05, 1, 1))
        fig.set_size_inches(8, 6, forward=True)
        fig.show()

    # Save model results to file
    if dump_pickle:
        model_results_dump = {}
        for parameter_name in model_results.keys():
            model_results_dump[parameter_name] = {}
            for result_type in model_results[parameter_name].keys():
                model_results_dump[parameter_name][result_type] = \
                    model_results[parameter_name][result_type].values

        pickle.dump(model_results_dump, open(path.join(
            path.curdir,
            f"{config_dict.get('file_name_base')}_model.p"), "wb"))

Fitting model to AES-128 Encrypt with 375 samples, from 16 to 6000 bytes in steps of 16.
[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 10
    # data points      = 375
    # variables        = 2
    chi-square         = 8.4145e-13
    reduced chi-square = 2.2559e-15
    Akaike info crit   = -12644.9647
    Bayesian info crit = -12637.1109
[[Variables]]
    intercept: -7.8313e-08 +/- 4.9152e-09 (6.28%) (init = 0)
    slope:      2.1572e-08 +/- 1.4161e-12 (0.01%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(intercept, slope) = -0.867
[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 375
    # variables        = 2
    chi-square         = 2.9742e-07
    reduced chi-square = 7.9738e-10
    Akaike info crit   = -7854.13939
    Bayesian info crit = -7846.28553
[[Variables]]
    intercept:  1.6185e-06 +/- 2.9226e-06 (180.58%) (init 

# Total from measurement

In [297]:
n_samples = 5995

total_energy = sum(aes_encrypt_energy[:n_samples]) + sum(aes_flash_write_energy[:n_samples]) + sum(aes_flash_read_energy[:n_samples]) + sum(aes_decrypt_energy[:n_samples])
total_time = sum(aes_encrypt_time_s[:n_samples]) + sum(aes_flash_write_time_s[:n_samples]) + sum(aes_flash_read_time_s[:n_samples]) + sum(aes_decrypt_time_s[:n_samples])
print(total_energy, total_time)

262.1258611546654 107160.84631253363


In [298]:
          [e + w + r + d for (e,w,r,d) in zip(aes_encrypt_energy, aes_flash_write_energy, aes_flash_read_energy, aes_decrypt_energy)],
          [e + w + r + d for (e,w,r,d) in zip(aes_encrypt_time_s, aes_flash_write_time_s, aes_flash_read_time_s, aes_decrypt_time_s)]

[3.630205333333338,
 4.818272533333334,
 6.2043509333333375,
 7.4584218666666615,
 9.24023520000003,
 10.559658666666666,
 11.879616000000038,
 13.133257066666655,
 14.849159999999973,
 16.171175999999953,
 17.425080533333357,
 18.743242666666717,
 20.526009066666628,
 21.71522826666672,
 23.099986666666737,
 24.353772800000172,
 27.853350400000032,
 29.302956800000057,
 30.556541866666564,
 31.877669866666622,
 33.59590026666659,
 34.912883733333345,
 36.234448533333236,
 37.554080533333206,
 39.20361173333331,
 40.459015466666855,
 41.77649066666667,
 43.22861120000021,
 44.81601653333334,
 46.19856533333345,
 47.455373333333156,
 48.772868266666386,
 52.27313226666675,
 53.589834666666555,
 54.97787306666657,
 56.29815893333334,
 57.94935359999975,
 59.19957493333361,
 60.457418666666875,
 61.83850186666718,
 63.62492906666573,
 64.87648213333364,
 66.19930773333316,
 67.4523008000003,
 69.16595733333341,
 70.55433599999938,
 71.67560320000011,
 73.19525386666737,
 76.69276373333389

Error in sys.excepthook:
Traceback (most recent call last):
  File "c:\users\areeb assad\appdata\local\programs\python\python38-32\lib\site-packages\ipykernel\kernelapp.py", line 171, in excepthook
    traceback.print_exception(etype, evalue, tb, file=sys.__stderr__)
  File "c:\users\areeb assad\appdata\local\programs\python\python38-32\lib\traceback.py", line 105, in print_exception
    print(line, file=file, end="")
OSError: [Errno 22] Invalid argument

Original exception was:
Traceback (most recent call last):
  File "c:\users\areeb assad\appdata\local\programs\python\python38-32\lib\site-packages\matplotlib\backends\backend_wx.py", line 1157, in _onClose
    Gcf.destroy(self.num)
  File "c:\users\areeb assad\appdata\local\programs\python\python38-32\lib\site-packages\matplotlib\_pylab_helpers.py", line 55, in destroy
    manager.destroy()
  File "c:\users\areeb assad\appdata\local\programs\python\python38-32\lib\site-packages\matplotlib\backends\backend_wx.py", line 1211, in destro